# More basic ETL

This week we'll take a look at more ETL functions, building up a mini warehouse using Bikeshare and weather data.

## Setup - install PostgreSQL (Optional)

We are going to use [PostgreSQL](https://www.postgresql.org) 9.5 or later version this time. If you are using AWS EC2 instances based on our AMI, you can skip this section. If postgresql is not installed, follow the [instructions](https://www.postgresql.org/download/linux/) to install it.

In order to connect to PostgreSQL, we need to make sure [ipython-sql](https://github.com/catherinedevlin/ipython-sql) and [psycopg2](https://github.com/psycopg/psycopg2) libraries are installed.

In [ ]:
!pip freeze | grep 'ipython-sql\|psycopg2'

If you see something like this, you are all set:
```
ipython-sql==0.3.8
psycopg2==2.6.2
```

## Setup - bikeshare data, again

We'll download the same Bikeshare data you've worked with before, and we'll create some database tables and indexes more deliberately using PostgreSQL.

In [ ]:
%load_ext sql

In [ ]:
!createdb -U student week11

In [ ]:
%sql postgresql://student@/week11

In [ ]:
!wget -O 2017-Q1-trips.zip https://s3.amazonaws.com/capitalbikeshare-data/2017-Q1-cabi-trips-history-data.zip

In [ ]:
!unzip -o 2017-Q1-trips.zip

In [ ]:
!mv 2017-Q1-Trips-History-Data.csv 2017q1.csv

In [ ]:
!wc -l 2017q1.csv

In [ ]:
!csvcut -n 2017q1.csv

### Create table and import

Given the volume of data here, let's go straight to pgsql to load the data.

*Note* use `gshuf` if you're on a Mac, otherwise try `shuf`.  Same options should work for both.

In [ ]:
!head -n 10000 2017q1.csv | csvstat

In [ ]:
!shuf -n 10000 2017q1.csv | csvstat

Based on these values, I expect we can work with the following:

In [ ]:
%%sql
DROP TABLE IF EXISTS rides;
CREATE TABLE rides (
    duration_ms INTEGER,
    start_date TIMESTAMP,
    end_date TIMESTAMP,
    start_station_id INTEGER,
    start_station VARCHAR(64),
    end_station_id INTEGER,
    end_station VARCHAR(64),
    bike_number CHAR(21),
    member_type CHAR(10)
)

Now we'll load the data directly using `COPY` command.  Note that this **requires** the use of an absolute path, so adjust it to your location:

In [ ]:
!pwd

In [ ]:
%%sql
COPY rides FROM '/home/ubuntu/star-week1/2017q1.csv'
CSV
HEADER;

In [ ]:
%%sql
SELECT COUNT(*) FROM rides;

In [ ]:
%%sql
SELECT * FROM rides
LIMIT 10

## More ETL with SQL

Today we'll extend previous week's examples of how to extract consistent sets of values out of your database.  

First let's pick up where we left off, extracting simple details like station names.

In [ ]:
%%sql
SELECT DISTINCT start_station, start_station_id
FROM rides
ORDER BY start_station
LIMIT 10;

In [ ]:
%%sql
SELECT DISTINCT end_station, end_station_id
FROM rides
ORDER BY end_station
LIMIT 10;

To be sure we get them all, we need to combine them into a union set.

In [ ]:
%%sql
SELECT DISTINCT start_station AS station, start_station_id AS station_id FROM rides
UNION
SELECT DISTINCT end_station AS station, end_station_id AS station_id FROM rides

Now we can create a new table to house the unique station names.

In [ ]:
%%sql
DROP TABLE IF EXISTS stations;
CREATE TABLE stations (
    key SERIAL PRIMARY KEY,
    name VARCHAR(64),
    station_key INTEGER
);

In [ ]:
%%sql
INSERT INTO stations (name, station_key)
SELECT DISTINCT start_station AS station, start_station_id AS station_key FROM rides
UNION
SELECT DISTINCT end_station AS station, end_station_id AS station_key FROM rides;

In [ ]:
%%sql
SELECT * FROM stations LIMIT 10;

We can even add these new identifiers back to the original table now.

In [ ]:
%%sql
ALTER TABLE rides 
ADD COLUMN start_station_key INTEGER,
ADD CONSTRAINT fk_start_station
FOREIGN KEY (start_station_key)
REFERENCES stations (key);

In [ ]:
%%sql
UPDATE rides AS r
SET start_station_key = s.key
FROM stations AS s
WHERE r.start_station = s.name;

In [ ]:
%%sql
ALTER TABLE rides 
ADD COLUMN end_station_key INTEGER,
ADD CONSTRAINT fk_end_station
FOREIGN KEY (end_station_key)
REFERENCES stations (key);

In [ ]:
%%sql
UPDATE rides AS r
SET end_station_key = s.key
FROM stations AS s
WHERE r.end_station = s.name;

In [ ]:
%%sql
SELECT * FROM rides
LIMIT 5;

### Simple address geocoding

It feels like we should do a little more with the stations, doesn't it?  Let's see if we can geocode them using the [geocoder library](https://geocoder.readthedocs.io/). First let's install `geocoder` package: 

In [ ]:
!sudo /home/ubuntu/.local/bin/pip install geocoder

In [ ]:
import geocoder

#### Connecting to the db from python

Here we'll use a little python to update run geocoding queries and flesh out the data a bit more.

In [ ]:
%%sql
ALTER TABLE stations
ADD COLUMN lat NUMERIC DEFAULT 0,
ADD COLUMN lng NUMERIC DEFAULT 0;

Note that the following code takes very long to finish. Please complete the following steps at home.

In [ ]:
import psycopg2

conn = psycopg2.connect("dbname='week11' user='student'")
c = conn.cursor()
c.execute("SELECT key, name FROM stations ORDER BY key ASC")
rows = c.fetchall()
for r in rows:
    station_key, station_name = r
    print('%s: %s' % (station_key, station_name))
    g = geocoder.google('%s Washington DC' % station_name)
    c.execute("UPDATE stations SET lat = (%s), lng = (%s) WHERE key = (%s)", 
              (g.lat, g.lng, station_key))
conn.commit()

In [ ]:
%%sql
SELECT AVG(lat), MIN(lat), MAX(lat), STDDEV(lat) FROM stations;

In [ ]:
%%sql
SELECT * FROM stations LIMIT 10 ;

In [ ]:
%%sql
SELECT COUNT(*) FROM stations WHERE lat IS NULL OR lng IS NULL;

Looks like it mostly worked.  It's a start, at least.

### Add more derived facts and dimensions

Another useful step might be recording the minutes as a new column so we don't have to calculate from milliseconds every time.

In [ ]:
%%sql
ALTER TABLE rides
ADD COLUMN duration_min NUMERIC;

In [ ]:
%%sql
UPDATE rides
SET duration_min = ROUND(CAST(duration_ms AS NUMERIC) / (1000 * 60), 1);

In [ ]:
%%sql
SELECT duration_ms, duration_min FROM rides
LIMIT 5;

In data warehouse models and in statistical model feature engineering, it can be particularly useful to extract all kinds of parts of dates out into their own attributes.  You never know where you'll find significance.

This kind of extraction is quite common.

In [ ]:
%%sql
SELECT DISTINCT TO_CHAR(start_date, 'YYYY-MM-DD HH24:00:00') AS hour,
    TO_CHAR(start_date, 'YYYY-MM-DD') AS day, 
    TO_CHAR(start_date, 'YYYY') AS year,
    TO_CHAR(start_date, 'MM') AS month_of_year,
    TO_CHAR(start_date, 'DD') AS day_of_month,
    TO_CHAR(start_date, 'Day') AS day_of_week_str,
    TO_CHAR(start_date, 'D') AS day_of_week,
    CASE WHEN CAST(TO_CHAR(start_date, 'D') AS INTEGER) >= 6 
        THEN 1 
        ELSE 0
    END AS is_weekend,
    CASE WHEN CAST(TO_CHAR(start_date, 'D') AS INTEGER) < 6 
        THEN 1 
        ELSE 0
    END AS is_weekday,
    TO_CHAR(start_date, 'HH24') AS hour_of_day,
    TO_CHAR(start_date, 'Q') AS quarter
FROM rides
LIMIT 10;

In [ ]:
%%sql
DROP TABLE IF EXISTS hours;
CREATE TABLE hours (
    key SERIAL PRIMARY KEY,
    hour CHAR(19),
    day CHAR(10),
    year INTEGER,
    month_of_year INTEGER,
    day_of_month INTEGER,
    day_of_week_str CHAR(9),
    day_of_week INTEGER,
    is_weekend BOOLEAN,
    is_weekday BOOLEAN,
    hour_of_day INTEGER,
    quarter INTEGER
);

In [ ]:
%%sql
INSERT INTO hours (hour, day, year, month_of_year, day_of_month, day_of_week_str, day_of_week,
                  is_weekend, is_weekday, hour_of_day, quarter)
SELECT DISTINCT TO_CHAR(start_date, 'YYYY-MM-DD HH24:00:00') AS hour,
    TO_CHAR(start_date, 'YYYY-MM-DD') AS day, 
    CAST(TO_CHAR(start_date, 'YYYY') AS INTEGER) AS year,
    CAST(TO_CHAR(start_date, 'MM') AS INTEGER) AS month_of_year,
    CAST(TO_CHAR(start_date, 'DD') AS INTEGER) AS day_of_month,
    TO_CHAR(start_date, 'Day') AS day_of_week_str,
    CAST(TO_CHAR(start_date, 'D') AS INTEGER) AS day_of_week,
    CASE WHEN CAST(TO_CHAR(start_date, 'D') AS INTEGER) IN (1, 7) 
        THEN TRUE
        ELSE FALSE
    END AS is_weekend,
    CASE WHEN CAST(TO_CHAR(start_date, 'D') AS INTEGER) NOT IN (1, 7) 
        THEN TRUE
        ELSE FALSE
    END AS is_weekday,
    CAST(TO_CHAR(start_date, 'HH24') AS INTEGER) AS hour_of_day,
    CAST(TO_CHAR(start_date, 'Q') AS INTEGER) AS quarter
FROM rides
UNION
SELECT DISTINCT TO_CHAR(end_date, 'YYYY-MM-DD HH24:00:00') AS hour,
    TO_CHAR(end_date, 'YYYY-MM-DD') AS day, 
    CAST(TO_CHAR(end_date, 'YYYY') AS INTEGER) AS year,
    CAST(TO_CHAR(end_date, 'MM') AS INTEGER) AS month_of_year,
    CAST(TO_CHAR(end_date, 'DD') AS INTEGER) AS day_of_month,
    TO_CHAR(end_date, 'Day') AS day_of_week_str,
    CAST(TO_CHAR(end_date, 'D') AS INTEGER) AS day_of_week,
    CASE WHEN CAST(TO_CHAR(end_date, 'D') AS INTEGER) IN (1, 7) 
        THEN TRUE
        ELSE FALSE
    END AS is_weekend,
    CASE WHEN CAST(TO_CHAR(end_date, 'D') AS INTEGER) NOT IN (1, 7) 
        THEN TRUE
        ELSE FALSE
    END AS is_weekday,
    CAST(TO_CHAR(end_date, 'HH24') AS INTEGER) AS hour_of_day,
    CAST(TO_CHAR(end_date, 'Q') AS INTEGER) AS quarter
FROM rides;

In [ ]:
%%sql
SELECT * FROM hours
LIMIT 10;

And let's make sure we got that weekend bit right:

In [ ]:
%%sql
SELECT DISTINCT day_of_week_str, day_of_week, is_weekend, is_weekday FROM hours;

In [ ]:
%%sql
ALTER TABLE rides 
ADD COLUMN start_hour_key INTEGER,
ADD CONSTRAINT fk_start_hour
FOREIGN KEY (start_hour_key)
REFERENCES hours (key);

In [ ]:
%%sql
UPDATE rides AS r
SET start_hour_key = h.key
FROM hours AS h
WHERE TO_CHAR(r.start_date, 'YYYY-MM-DD HH24:00:00') = h.hour;

In [ ]:
%%sql
ALTER TABLE rides 
ADD COLUMN end_hour_key INTEGER,
ADD CONSTRAINT fk_end_hour
FOREIGN KEY (end_hour_key)
REFERENCES hours (key);

In [ ]:
%%sql
UPDATE rides AS r
SET end_hour_key = h.key
FROM hours AS h
WHERE TO_CHAR(r.end_date, 'YYYY-MM-DD HH24:00:00') = h.hour;

In [ ]:
%%sql
SELECT rides.start_date, rides.end_date, s_hours.hour AS start_hour, e_hours.hour AS end_hour
FROM rides
JOIN hours AS s_hours
  ON s_hours.key = rides.start_hour_key
JOIN hours AS e_hours
  ON e_hours.key = rides.end_hour_key
WHERE bike_number = 'W20893'
ORDER BY rides.start_date
LIMIT 10;

Now we are ready to explore the data. Let's find out the number of trips for each day of the week:

In [ ]:
%%sql
SELECT day_of_week_str, COUNT(*) count
FROM rides, hours
WHERE rides.start_hour_key = hours.key
GROUP BY day_of_week_str, day_of_week
ORDER BY day_of_week;

In [ ]:
%matplotlib inline

In [ ]:
result = _
result.bar()

## Adding weather data

An interesting dimension to the bikeshare history is weather - I know I don't mind riding in the rain, but I'm probably the only one.

Weather Underground offers access to weather history data at links like https://www.wunderground.com/history/airport/KDCA/2017/1/18/DailyHistory.html?req_city=Washington&req_state=DC&req_statename=District+of+Columbia&reqdb.zip=20003&reqdb.magic=1&reqdb.wmo=99999.  

You can also download data in CSV format. For example: https://www.wunderground.com/history/airport/KDCA/2017/1/18/DailyHistory.html?req_city=Washington&req_state=DC&req_statename=District+of+Columbia&reqdb.zip=20003&reqdb.magic=1&reqdb.wmo=99999&format=1

Mmm, CSV.  We know what to do with CSV.

In [ ]:
from string import Template
import requests

In [ ]:
url_template = Template('https://www.wunderground.com/history/airport/KDCA/$year/$month/$day/DailyHistory.html?req_city=Washington&req_state=DC&req_statename=District+of+Columbia&reqdb.zip=20003&reqdb.magic=1&reqdb.wmo=99999&format=1')
print(url_template.substitute(year=2017, month=1, day=18))

Let's write simple python code to download the weather data for the first quarter of 2017.

In [ ]:
import calendar
year = 2017
for month in range(1, 4):
    days = calendar.monthrange(year, month)[1]
    for day in range(1, days+1):
        r = requests.get(url_template.substitute(year=year, month=month, day=day))
        print('Saving weather-%04d%02d%02d.csv' % (year, month, day))
        open('weather-%04d%02d%02d.csv' % (year, month, day), 'wb').write(r.content)

Alternatively you can download these csv files as a zip file and unzip:

In [ ]:
!wget -O weather2017q1.csv.zip https://raw.githubusercontent.com/tongwang/data2017/master/lectures/star-week-01/weather2017q1.csv.zip
!unzip -o weather2017q1.csv.zip

In [ ]:
!head weather-20170125.csv | csvlook

Something is not right! The header is missing and there is `<br />` at the end of each line. Let's look at the raw content of the CSV file.

In [ ]:
!head weather-20170125.csv

There are two issues:
1. The first line is blank
2. There are extra characters at the end of each line.

Let's clean the data:

In [ ]:
!sed 's/<br \/>//g;/^$/d' weather-20170125.csv | head | csvlook

Now it looks much better! Apply the fix to all weather CSV files.

In [ ]:
!for f in weather-2017*.csv; do sed -i 's/<br \/>//g;/^$/d' ${f}; done

And combine date based csv files into a single file:

In [ ]:
!csvstack weather-201701*.csv weather-201702*.csv weather-201703*.csv > weather-2017q1.csv

In [ ]:
!csvstat weather-2017q1.csv

We've noticed special values such as `N/A`, `-` and `None`. We need to remove them so that they will be treated as NULL by the database.

In [ ]:
!sed -i 's/,N\/A,/,,/g;s/,-,/,,/g;;s/,None,/,,/g' weather-2017q1.csv

Based on these values, I expect we can work with the following schema for weather. Note that the type for `time_utc` is `TIMESTAMPTZ`, which is the abbreviation for `timestamp with time zone`, a PostgreSQL specific type. We also add an attribute named `time`. We will use it to store local eastern time.

In [ ]:
%%sql
DROP TABLE IF EXISTS weather;
CREATE TABLE weather (
    key SERIAL PRIMARY KEY,
    time_str VARCHAR(8),
    temp NUMERIC,
    dew_point NUMERIC,
    humidity NUMERIC,
    pressure NUMERIC,
    visibility NUMERIC,
    wind_dir VARCHAR(8),
    wind_speed VARCHAR(10),
    gust_speed NUMERIC,
    precipitation NUMERIC,
    events VARCHAR(50),
    conditions VARCHAR(50),
    wind_dir_degrees NUMERIC,
    time_utc TIMESTAMPTZ,
    time TIMESTAMP
)

Now we'll load the data into PostgreSQL. Note that this requires the use of an absolute path, so adjust it to your location:

In [ ]:
!pwd

In [ ]:
%%sql
COPY weather 
(time_str, temp, dew_point, humidity, pressure, visibility, wind_dir, wind_speed, gust_speed, precipitation, events, conditions, wind_dir_degrees, time_utc)
FROM '/home/ubuntu/star-week1/weather-2017q1.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

In [ ]:
%%sql
SELECT * from weather LIMIT 10;

Next, we need to convert UTC time to EST or EDT. We know Daylight Saving Time started on Sunday, March 12, 2017, 2:00:00 am. The conversion takes two steps:

First we convert UTC times to EST times and populate `time` attribute for all `time_utc` values before `2017-03-12 07:00:00+00:00`, which is Sunday, March 12, 2017, 2:00:00 am EST. 

In [ ]:
%%sql
UPDATE weather SET time = time_utc AT TIME ZONE 'EST'
WHERE time_utc <= '2017-03-12 07:00:00+00:00';

Next we convert UTC times to EDT times and populate `time` attribute for all `time_utc` values after `2017-03-12 07:00:00+00:00`. 

In [ ]:
%%sql
UPDATE weather SET time = time_utc AT TIME ZONE 'EDT'
WHERE time_utc > '2017-03-12 07:00:00+00:00';

Verify that time attributes look okay on March 12:

In [ ]:
%%sql
SELECT time_str, time from weather 
WHERE TO_CHAR(time, 'YYYY-MM-DD') = '2017-03-12'
ORDER BY time;

Now we add two foreign key columns (`start_weather_key` and `end_weather_key`) to the `rides` table that reference `weather` dimension table.

In [ ]:
%%sql
ALTER TABLE rides 
ADD COLUMN start_weather_key INTEGER,
ADD CONSTRAINT fk_start_weather
FOREIGN KEY (start_weather_key)
REFERENCES weather (key);

In [ ]:
%%sql
ALTER TABLE rides 
ADD COLUMN end_weather_key INTEGER,
ADD CONSTRAINT fk_end_weather
FOREIGN KEY (end_weather_key)
REFERENCES weather (key);

In [ ]:
%%sql
UPDATE rides AS r
SET start_weather_key = w.key
FROM weather AS w
WHERE TO_CHAR(r.start_date, 'YYYY-MM-DD HH24') = TO_CHAR(w.time, 'YYYY-MM-DD HH24');

In [ ]:
%%sql
UPDATE rides AS r
SET end_weather_key = w.key
FROM weather AS w
WHERE TO_CHAR(r.end_date, 'YYYY-MM-DD HH24') = TO_CHAR(w.time, 'YYYY-MM-DD HH24');

Some rides do not have weather captured because there are some missing hours in the weather data:

In [ ]:
%%sql
SELECT COUNT(*) FROM rides
WHERE start_weather_key IS NULL;

In [ ]:
%%sql
SELECT COUNT(*) FROM rides
WHERE end_weather_key IS NULL;

Let's find out the top 5 weather conditions that people ride bikeshare.

In [ ]:
%%sql
SELECT w.conditions, COUNT(*) count
FROM rides
JOIN weather AS w
ON w.key = rides.start_weather_key
GROUP BY w.conditions
ORDER BY count DESC
LIMIT 5;

In [ ]:
%matplotlib inline

In [ ]:
result = _
result.bar()

And the top 10 weather conditions that people ride bikeshare the least often:

In [ ]:
%%sql
SELECT w.conditions, COUNT(*) count
FROM rides
JOIN weather AS w
ON w.key = rides.start_weather_key
GROUP BY w.conditions
ORDER BY count ASC
LIMIT 10;

In [ ]:
result = _
result.bar()